In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", context="talk")

In [2]:
train = pd.read_csv('favorita-grocery-sales-forecasting/train.csv/train.csv')
print(train.head())

C:\Users\Dell\AppData\Local\Temp\ipykernel_6704\2374804111.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('favorita-grocery-sales-forecasting/train.csv/train.csv')


   id        date  store_nbr  item_nbr  unit_sales onpromotion
0   0  2013-01-01         25    103665         7.0         NaN
1   1  2013-01-01         25    105574         1.0         NaN
2   2  2013-01-01         25    105575         2.0         NaN
3   3  2013-01-01         25    108079         1.0         NaN
4   4  2013-01-01         25    108701         1.0         NaN


In [3]:
test_path = 'favorita-grocery-sales-forecasting/test.csv/test.csv'
transactions_path = 'favorita-grocery-sales-forecasting/transactions.csv/transactions.csv'
holidays_path = 'favorita-grocery-sales-forecasting/holidays_events.csv/holidays_events.csv'
items_path = 'favorita-grocery-sales-forecasting/items.csv/items.csv'
oil_path = 'favorita-grocery-sales-forecasting/oil.csv/oil.csv'
stores_path = 'favorita-grocery-sales-forecasting/stores.csv/stores.csv'

In [4]:
# Load the datasets
test = pd.read_csv(test_path)
transactions = pd.read_csv(transactions_path)
holidays = pd.read_csv(holidays_path)
items = pd.read_csv(items_path)
oil = pd.read_csv(oil_path)
stores = pd.read_csv(stores_path)

In [5]:
print("Train:", train.shape)
print("Test:", test.shape)
print("Transactions:", transactions.shape)
print("Holidays:", holidays.shape)
print("Items:", items.shape)
print("Oil:", oil.shape)
print("Stores:", stores.shape)

Train: (125497040, 6)
Test: (3370464, 5)
Transactions: (83488, 3)
Holidays: (350, 6)
Items: (4100, 4)
Oil: (1218, 2)
Stores: (54, 5)


In [6]:
# Convert the 'date' column to datetime
train['date'] = pd.to_datetime(train['date'], format='%Y-%m-%d')

In [7]:
print("Train Data Info:")
print(train.info())
print(train.head())

Train Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   item_nbr     int64         
 4   unit_sales   float64       
 5   onpromotion  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 5.6+ GB
None
   id       date  store_nbr  item_nbr  unit_sales onpromotion
0   0 2013-01-01         25    103665         7.0         NaN
1   1 2013-01-01         25    105574         1.0         NaN
2   2 2013-01-01         25    105575         2.0         NaN
3   3 2013-01-01         25    108079         1.0         NaN
4   4 2013-01-01         25    108701         1.0         NaN


In [8]:
unique_values = train['onpromotion'].unique()
print("Unique values in onpromotion:", unique_values)

Unique values in onpromotion: [nan False True]


In [9]:
promotion_counts = train['onpromotion'].value_counts()
print("Counts of onpromotion values:")
print(promotion_counts)

Counts of onpromotion values:
False    96028767
True      7810622
Name: onpromotion, dtype: int64


In [10]:
train['onpromotion'] = train['onpromotion'].fillna(False).astype('bool')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   item_nbr     int64         
 4   unit_sales   float64       
 5   onpromotion  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3)
memory usage: 4.8 GB


In [11]:
# Basic describe
print("Basic Statistics:\n", train['unit_sales'].describe())

Basic Statistics:
 count    1.254970e+08
mean     8.554865e+00
std      2.360515e+01
min     -1.537200e+04
25%      2.000000e+00
50%      4.000000e+00
75%      9.000000e+00
max      8.944000e+04
Name: unit_sales, dtype: float64


In [ ]:
# IQR method for positive sales
q1 = train['unit_sales'].quantile(0.25)
q3 = train['unit_sales'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr
 
# Identify outliers
outliers = train[(train['unit_sales'] < lower_bound) | (train['unit_sales'] > upper_bound)]

# Count of negative sales
negative_sales = train[train['unit_sales'] < 0]

In [13]:
print(f"\nNumber of outliers: {len(outliers):,}")
print(f"Number of negative unit_sales: {len(negative_sales):,}")


Number of outliers: 11,497,596
Number of negative unit_sales: 7,795


In [14]:
# Remove negative unit_sales
train_cleaned = train[train['unit_sales'] >= 0]
 
# Remove outliers based on IQR
train_cleaned = train_cleaned[(train_cleaned['unit_sales'] >= lower_bound) &
                              (train_cleaned['unit_sales'] <= upper_bound)]
 
print(f"Cleaned DataFrame shape: {train_cleaned.shape}")
train_cleaned.info()

Cleaned DataFrame shape: (113992814, 6)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 113992814 entries, 0 to 125497039
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   item_nbr     int64         
 4   unit_sales   float64       
 5   onpromotion  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(3)
memory usage: 5.2 GB


In [15]:
train=train_cleaned.copy()

In [16]:
# Ensure key columns have proper types
stores['store_nbr'] = stores['store_nbr'].astype(int)
items['item_nbr'] = items['item_nbr'].astype(int)

In [17]:
print("\nStores Data:")
print(stores.head())
print("\nItems Data:")
print(items.head())


Stores Data:
   store_nbr           city                           state type  cluster
0          1          Quito                       Pichincha    D       13
1          2          Quito                       Pichincha    D       13
2          3          Quito                       Pichincha    D        8
3          4          Quito                       Pichincha    D        9
4          5  Santo Domingo  Santo Domingo de los Tsachilas    D        4

Items Data:
   item_nbr        family  class  perishable
0     96995     GROCERY I   1093           0
1     99197     GROCERY I   1067           0
2    103501      CLEANING   3008           0
3    103520     GROCERY I   1028           0
4    103665  BREAD/BAKERY   2712           1


In [18]:
# Merge store information on 'store_nbr'
merged_df = train.merge(stores, on='store_nbr', how='inner')
# Merge item information on 'item_nbr'
merged_df = merged_df.merge(items, on='item_nbr', how='inner')
print("\nAfter Merging Stores and Items:")
print(merged_df.head())
print(merged_df.info())


After Merging Stores and Items:
       id       date  store_nbr  item_nbr  unit_sales  onpromotion     city  \
0       0 2013-01-01         25    103665         7.0        False  Salinas   
1   19811 2013-01-02         25    103665         5.0        False  Salinas   
2  100696 2013-01-04         25    103665         5.0        False  Salinas   
3  141831 2013-01-05         25    103665         5.0        False  Salinas   
4  183815 2013-01-06         25    103665         7.0        False  Salinas   

         state type  cluster        family  class  perishable  
0  Santa Elena    D        1  BREAD/BAKERY   2712           1  
1  Santa Elena    D        1  BREAD/BAKERY   2712           1  
2  Santa Elena    D        1  BREAD/BAKERY   2712           1  
3  Santa Elena    D        1  BREAD/BAKERY   2712           1  
4  Santa Elena    D        1  BREAD/BAKERY   2712           1  
<class 'pandas.core.frame.DataFrame'>
Int64Index: 113992814 entries, 0 to 113992813
Data columns (total 13 c

In [19]:
# Compute average unit sales per item (or use groupby(['store_nbr','item_nbr']) for a more granular metric)
avg_sales = merged_df.groupby('item_nbr')['unit_sales'].mean().reset_index()
avg_sales.rename(columns={'unit_sales': 'avg_unit_sales'}, inplace=True)
# Merge avg_unit_sales back into the main DataFrame
merged_df = merged_df.merge(avg_sales, on='item_nbr', how='inner')
print("\nAfter Computing and Merging Average Unit Sales:")
print(merged_df[['item_nbr', 'avg_unit_sales']].head())


After Computing and Merging Average Unit Sales:
   item_nbr  avg_unit_sales
0    103665        4.254618
1    103665        4.254618
2    103665        4.254618
3    103665        4.254618
4    103665        4.254618


In [20]:
# Select only the required columns
final_columns = [
    'id', 'date', 'store_nbr', 'item_nbr', 'unit_sales', 'onpromotion',
    'city', 'state', 'type', 'cluster', 'family', 'class', 'perishable',
    'avg_unit_sales'
]
final_dataset = merged_df[final_columns]
print("\nFinal Dataset Preview:")
print(final_dataset.head())


Final Dataset Preview:
       id       date  store_nbr  item_nbr  unit_sales  onpromotion     city  \
0       0 2013-01-01         25    103665         7.0        False  Salinas   
1   19811 2013-01-02         25    103665         5.0        False  Salinas   
2  100696 2013-01-04         25    103665         5.0        False  Salinas   
3  141831 2013-01-05         25    103665         5.0        False  Salinas   
4  183815 2013-01-06         25    103665         7.0        False  Salinas   

         state type  cluster        family  class  perishable  avg_unit_sales  
0  Santa Elena    D        1  BREAD/BAKERY   2712           1        4.254618  
1  Santa Elena    D        1  BREAD/BAKERY   2712           1        4.254618  
2  Santa Elena    D        1  BREAD/BAKERY   2712           1        4.254618  
3  Santa Elena    D        1  BREAD/BAKERY   2712           1        4.254618  
4  Santa Elena    D        1  BREAD/BAKERY   2712           1        4.254618  


In [21]:
# Export the Final Dataset as CSV
# final_csv_path = '/content/drive/My Drive/YourFolderName/final_train_dataset1.csv'
# final_dataset.to_csv(final_csv_path, index=False)
# print("\n✅ Final training dataset saved to:", final_csv_path)

In [22]:
# Step 1: Make sure 'date' is datetime
final_dataset['date'] = pd.to_datetime(final_dataset['date'])
 
# Step 2: Filter records from 2015 onwards
filtered_df = final_dataset[final_dataset['date'] >= '2016-01-01']
 
# Step 3: Show memory usage summary
memory_usage = filtered_df.memory_usage(deep=True)
total_memory_mb = memory_usage.sum() / (1024 ** 2)
 
# Step 4: Print memory details
print("Memory usage by column:")
print(memory_usage)
 
print(f"\nTotal memory usage: {total_memory_mb:.2f} MB")
 
# Optional: Show shape to confirm size
print(f"\nFiltered records: {filtered_df.shape[0]} rows, {filtered_df.shape[1]} columns")

Memory usage by column:
Index              432506336
id                 432506336
date               432506336
store_nbr          432506336
item_nbr           432506336
unit_sales         432506336
onpromotion         54063292
city              3442851049
state             3565259079
type              3135670936
cluster            432506336
family            3553130020
class              432506336
perishable         432506336
avg_unit_sales     432506336
dtype: int64

Total memory usage: 17238.65 MB

Filtered records: 54063292 rows, 14 columns


In [23]:
filtered_df = filtered_df.drop(columns=['store_nbr', 'item_nbr'])

# Keep only the records where the 'id' column has non-null values.
filtered_df = filtered_df[filtered_df['id'].notnull()]

# Optionally, reset the index after filtering.
final_dataset = filtered_df.reset_index(drop=True)

# To check the result:
print(final_dataset.head())

         id       date  unit_sales  onpromotion     city        state type  \
0  66894873 2016-01-06         2.0        False  Salinas  Santa Elena    D   
1  67077674 2016-01-08         4.0        False  Salinas  Santa Elena    D   
2  67173081 2016-01-09         6.0        False  Salinas  Santa Elena    D   
3  67549594 2016-01-13         1.0        False  Salinas  Santa Elena    D   
4  67638605 2016-01-14         2.0        False  Salinas  Santa Elena    D   

   cluster        family  class  perishable  avg_unit_sales  
0        1  BREAD/BAKERY   2712           1        4.254618  
1        1  BREAD/BAKERY   2712           1        4.254618  
2        1  BREAD/BAKERY   2712           1        4.254618  
3        1  BREAD/BAKERY   2712           1        4.254618  
4        1  BREAD/BAKERY   2712           1        4.254618  


In [24]:
memory_usage = final_dataset.memory_usage(deep=True)
total_memory_mb = memory_usage.sum() / (1024 ** 2)
 
# Step 4: Print memory details
print("Memory usage by column:")
print(memory_usage)

Memory usage by column:
Index                    128
id                 432506336
date               432506336
unit_sales         432506336
onpromotion         54063292
city              3442851049
state             3565259079
type              3135670936
cluster            432506336
family            3553130020
class              432506336
perishable         432506336
avg_unit_sales     432506336
dtype: int64


In [25]:
print(f"\nTotal memory usage: {total_memory_mb:.2f} MB")


Total memory usage: 16001.24 MB


In [26]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54063292 entries, 0 to 54063291
Data columns (total 12 columns):
 #   Column          Dtype         
---  ------          -----         
 0   id              int64         
 1   date            datetime64[ns]
 2   unit_sales      float64       
 3   onpromotion     bool          
 4   city            object        
 5   state           object        
 6   type            object        
 7   cluster         int64         
 8   family          object        
 9   class           int64         
 10  perishable      int64         
 11  avg_unit_sales  float64       
dtypes: bool(1), datetime64[ns](1), float64(2), int64(4), object(4)
memory usage: 4.5+ GB


In [28]:
# Export the Final Dataset as CSV
final_dataset.to_csv('final_dataset.csv', index=False)